In [1]:
## crawler
import ru_dictionary_parser

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException

import pandas as pd

## 1. crawl openrussian dictionary

In [2]:
url_list = ["https://en.openrussian.org/list/all"+'/?start='+str(page_num) for page_num in range(50,90050,50)]
url_list.insert(0,"https://en.openrussian.org/list/all")

In [ ]:
## getting word lists
word_links_list = []

## click next page
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

for main_url in url_list:
    driver.get(main_url)

    anchor_tags = driver.find_elements(By.CSS_SELECTOR,'table a')
    word_links = [a.get_attribute('href') for a in anchor_tags]
    word_links_list.extend(word_links)

driver.close()

In [4]:
## get word data
word_data = ru_dictionary_parser.word_parser(word_links_list)

IndexError: list index out of range

## 2. make dictionary dataframe

In [ ]:
## make data tables

word_title_df = pd.DataFrame(word_data[0], columns = ['canonical_id', 'canonical_form'])
word_pos_df = pd.DataFrame(word_data[1], columns = ['canonical_id', 'pos'])
word_meaning_df = pd.DataFrame(word_data[2], columns = ['canonical_id', 'meaning'])
word_declension_df_ = pd.DataFrame(word_data[3], columns = ['canonical_id', 'tags', 'form'])
word_aspect_df = pd.DataFrame(word_data[4], columns = ['canonical_id', 'verb_aspect'])
word_imperfective_perfective_df = pd.DataFrame(word_data[5], columns = ['canonical_id', 'imperfective_perfective'])

In [ ]:
## word_canonical_df

word_canonical_df = word_title_df\
                    .merge(word_pos_df, how='left',on='canonical_id')\
                    .merge(word_meaning_df, how='left',on='canonical_id')\
                    .merge(word_aspect_df, how='left',on='canonical_id')\
                    .merge(word_imperfective_perfective_df, how='left',on='canonical_id')

word_canonical_df['is_saved'] = 0

word_canonical_df

In [ ]:
## word_declension_df

## clean_form column
def strip_stress_marks(text):
    b = text.encode('utf-8')
   # correct error where latin accented ó is used
    b = b.replace(b'\xc3\xb3', b'\xd0\xbe')
   # correct error where latin accented á is used
    b = b.replace(b'\xc3\xa1', b'\xd0\xb0')
   # correct error where latin accented é is used
    b = b.replace(b'\xc3\xa0', b'\xd0\xb5')
   # correct error where latin accented ý is used
    b = b.replace(b'\xc3\xbd', b'\xd1\x83')
   # remove combining diacritical mark
    b = b.replace(b'\xcc\x81',b'').decode()
    return b

In [ ]:
## word_declension_df
word_declension_df = word_declension_df_.copy()
word_declension_df['declension_id'] = list(range(word_declension_df.shape[0]))
word_declension_df['clean_form'] = [strip_stress_marks(f) for f in word_declension_df.form]

word_declension_df = word_declension_df[['declension_id', 'form', 'clean_form', 'tags', 'canonical_id']]

In [ ]:
word_declension_df

## 4. DB save

In [ ]:
########## 격변화 테이블 "-" 데이터 제거